In [62]:
from bs4 import BeautifulSoup
import json
import requests
import polars as pl
from shared.utils import formalize_data
import datetime

In [63]:
df_cw_gn = pl.read_parquet("./coin_walk_db_gn.parquet")
df_cw_kbl = pl.read_parquet("./coin_walk_db_kbl.parquet")
df_cw_tnk = pl.read_parquet("./coin_walk_db_tnk.parquet")

print(f'{len(df_cw_gn)=} | cluster_size={len(df_cw_gn)/1024}')
print(f'{len(df_cw_kbl)=} | cluster_size={len(df_cw_kbl)/1024}')
print(f'{len(df_cw_tnk)=} | cluster_size={len(df_cw_tnk)/1024}')

len(df_cw_gn)=324632 | cluster_size=317.0234375
len(df_cw_kbl)=3838380 | cluster_size=3748.41796875
len(df_cw_tnk)=15890700 | cluster_size=15518.26171875


In [64]:
df_cw_gn[0]


cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,bolillasid
str,str,str,str,str,str,str,str,str,str,str
"""A""","""A1""","""A11""","""A111""","""A1111""","""A11111""","""A111111""","""A1111111""","""A11111111""","""A111111111""","""0102030405"""


In [65]:

url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')



In [66]:
lst = []
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  cols[0] = (datetime.datetime.strptime(cols[0], "%d/%m/%Y").strftime("%Y-%m-%d"))
  id, bolillas_int = formalize_data(bolillas_str=cols[2])
  cols.append(id)
  
  lst.append(cols)

df_gn = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'bolillasid'
  ],
  orient='row'
)

TOTAL_LINES = len(df_gn)
print(f'{TOTAL_LINES=}')
display(df_gn.head(10))

TOTAL_LINES=7709


fecha,sorteo,bolillas,bolillasid
str,str,str,str
"""2025-03-24""","""4165""","""32 29 27 21 06""","""0621272932"""
"""2025-03-23""","""4164""","""13 10 04 15 08""","""0408101315"""
"""2025-03-22""","""4163""","""19 32 17 10 22""","""1017192232"""
"""2025-03-21""","""4162""","""32 33 01 17 26""","""0117263233"""
"""2025-03-20""","""4161""","""32 24 17 12 27""","""1217242732"""
"""2025-03-19""","""4160""","""01 13 07 22 14""","""0107131422"""
"""2025-03-18""","""4159""","""07 34 16 12 21""","""0712162134"""
"""2025-03-17""","""4158""","""20 30 04 08 24""","""0408202430"""
"""2025-03-16""","""4157""","""26 10 21 16 30""","""1016212630"""


In [67]:

df_clustered = df_gn.join(df_cw_gn, on="bolillasid")

In [68]:

df_clustered = df_clustered.sort('fecha', descending=True)
df_clustered.write_excel("./coin_walk_gn.xlsx")
df_clustered[0:10]

fecha,sorteo,bolillas,bolillasid,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2025-03-24""","""4165""","""32 29 27 21 06""","""0621272932""","""B""","""B1""","""B11""","""B111""","""B1111""","""B11111""","""B111111""","""B1111111""","""B11111111""","""B111111111"""
"""2025-03-23""","""4164""","""13 10 04 15 08""","""0408101315""","""B""","""B1""","""B12""","""B122""","""B1222""","""B12221""","""B122212""","""B1222122""","""B12221221""","""B122212212"""
"""2025-03-22""","""4163""","""19 32 17 10 22""","""1017192232""","""A""","""A2""","""A21""","""A211""","""A2111""","""A21111""","""A211112""","""A2111122""","""A21111221""","""A211112211"""
"""2025-03-21""","""4162""","""32 33 01 17 26""","""0117263233""","""A""","""A1""","""A11""","""A112""","""A1122""","""A11221""","""A112212""","""A1122122""","""A11221221""","""A112212212"""
"""2025-03-20""","""4161""","""32 24 17 12 27""","""1217242732""","""A""","""A2""","""A22""","""A222""","""A2222""","""A22222""","""A222221""","""A2222212""","""A22222122""","""A222221222"""
"""2025-03-19""","""4160""","""01 13 07 22 14""","""0107131422""","""B""","""B2""","""B21""","""B211""","""B2112""","""B21121""","""B211211""","""B2112112""","""B21121121""","""B211211211"""
"""2025-03-18""","""4159""","""07 34 16 12 21""","""0712162134""","""B""","""B1""","""B12""","""B121""","""B1212""","""B12122""","""B121222""","""B1212221""","""B12122211""","""B121222112"""
"""2025-03-17""","""4158""","""20 30 04 08 24""","""0408202430""","""B""","""B1""","""B12""","""B122""","""B1222""","""B12221""","""B122211""","""B1222112""","""B12221122""","""B122211222"""
"""2025-03-16""","""4157""","""26 10 21 16 30""","""1016212630""","""A""","""A2""","""A22""","""A222""","""A2222""","""A22221""","""A222212""","""A2222121""","""A22221212""","""A222212121"""


In [80]:
def show_b_stats (col: str, df: pl.DataFrame):
  df_viz = df.group_by(
    col
  ).agg(pl.count(col).alias('count')).sort('count', descending=True)

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)

  return df_viz

In [70]:
show_b_stats('cluster_1', df_clustered)

cluster_1,count
str,u32
"""A""",3871
"""B""",3838


alt.Chart(...)

In [71]:
show_b_stats('cluster_2', df_clustered)

cluster_2,count
str,u32
"""A2""",1938
"""B2""",1937
"""A1""",1933
"""B1""",1901


alt.Chart(...)

In [72]:
show_b_stats('cluster_3', df_clustered)

cluster_3,count
str,u32
"""A11""",1011
"""B21""",1000
"""A22""",976
"""B11""",975
"""A21""",962
"""B22""",937
"""B12""",926
"""A12""",922


alt.Chart(...)

In [73]:
show_b_stats('cluster_4', df_clustered)

cluster_4,count
str,u32
"""A212""",523
"""A111""",515
"""A222""",502
"""B212""",502
"""B111""",499
…,…
"""A121""",463
"""A122""",459
"""B222""",454


alt.Chart(...)

In [74]:
show_b_stats('cluster_5', df_clustered)

cluster_5,count
str,u32
"""B2122""",265
"""A2121""",262
"""A2221""",261
"""A1121""",261
"""A1112""",261
…,…
"""B1212""",223
"""B1211""",222
"""B2221""",222


alt.Chart(...)

In [75]:
show_b_stats('cluster_6', df_clustered)

cluster_6,count
str,u32
"""A11122""",153
"""A21221""",139
"""B12211""",138
"""B21221""",137
"""B11222""",136
…,…
"""A21121""",102
"""B12121""",101
"""A21112""",98


alt.Chart(...)

In [76]:
show_b_stats('cluster_7', df_clustered)

cluster_7,count
str,u32
"""A112122""",84
"""A111221""",82
"""B111112""",78
"""B112112""",76
"""A222112""",76
…,…
"""B112111""",45
"""B122122""",44
"""A211122""",43


alt.Chart(...)

In [77]:
show_b_stats('cluster_8', df_clustered)

cluster_8,count
str,u32
"""A1112212""",47
"""A1121222""",44
"""A2221122""",44
"""B1112111""",44
"""B1111211""",44
…,…
"""A1222121""",18
"""A1122221""",17
"""B2212112""",16


alt.Chart(...)

In [78]:
show_b_stats('cluster_9', df_clustered)

cluster_9,count
str,u32
"""B22122111""",26
"""B11112111""",25
"""A11122122""",25
"""B22111221""",24
"""B11121112""",24
…,…
"""A22121112""",7
"""B11112211""",6
"""A11221121""",6


alt.Chart(...)

In [81]:
test = show_b_stats('cluster_10', df_clustered)
test.write_excel("./gn_cluster10.xlsx")

cluster_10,count
str,u32
"""A221222121""",16
"""A111221222""",16
"""A221211221""",15
"""B221221111""",15
"""B211221211""",15
…,…
"""B122122212""",2
"""B222221111""",2
"""B222211221""",1


alt.Chart(...)